# 1.0 Filter data used in vacancy model based on TCI parcel numbers

Parcels were surveyed in Summer 2015, so all data pulled should come from before the inspection dates. Goal of script/notebook is to filter datasets by the parcel numbers in the TCI survey, although we will filter again based on existence of structure.

In [3]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
%matplotlib inline

path = '/'.join(os.getcwd().split('/')[:-2])
print(path)

/Volumes/Dropbox/largetransfer/luc/carter


//anaconda/envs/data_sci/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
# OLD CODE THAT READ EXCEL SPREADSHEET AND EXPORTED TO CSV
#
# tci = pd.read_excel(path+'/data/inspection_data/Cleveland_Final_Results_Table_FOR_DISTRIBUTION_20151111.xlsx', encoding="ISO-8859-1") 
# def get_vacant(x):
#     if x == 'Occupied Structure':
#         return 0
#     elif x == 'Vacant Structure':
#         return 1
#     else: 
#         return -1
# tci['vacant'] = tci['Survey Category'].apply(get_vacant)
# tci['parcel'] = tci.PIN.apply(lambda x: x[0:3]+'-'+x[3:5]+'-'+x[5:])
# # tci[(tci.USE_CLASS=='R') & (tci.vacant>-1)].to_csv(path+'/data/model_data/tci_1_0.csv', index=False)

tci = pd.read_csv(path+'/data/model_data/tci_1_0.csv', parse_dates=['Date'], dtype={'Ward':object,'PIN':str})
ppns = set(tci[(tci.USE_CLASS=='R') & (tci.vacant>-1)].parcel)

In [5]:
# how many vacant homes did they find?
print(sum(tci[tci.vacant>0].vacant))
# how many residential properties are there?
print(tci[(tci.USE_CLASS=='R')].shape)

10038

### Demographic data

In [ ]:
# NOT INCLUDED! Tested, but did not improve the model

# tracts = pd.read_csv(path+'/data/original_data/clv_par_census.csv')
# demo = pd.read_csv(path+'/data/original_data/sociodemographic_Data.csv')

# tracts = pd.merge(tracts, demo, left_on='NAME10', right_on='Census Tract', how='left')
# cols = [0,1,5,7,9,11,13,14,15,16,17,18,20,22,24,26,28,30,32,34,36,38,40]
# tracts.iloc[:,cols].to_csv(path+'/data/clean_data/demographic.csv',index=False)

### Property characteristics

In [5]:
# from neighborhood stuff?
# import simpledbf
# dbf = simpledbf.Dbf5(path+'/data/original_data/parcel0611_lookup2010.dbf').to_dataframe()

In [ ]:
# infile = path+'/data/original_data/main_prop.csv'
# outfile = path+'/data/clean_data/main_prop13.csv'

# with open(infile, 'r', encoding="ISO-8859-1") as fin, open(outfile, 'w') as fout:
#     write_to = csv.writer(fout, lineterminator='\n')
#     header = next(csv.reader(fin))
#     write_to.writerow(header)
#     for row in csv.reader(fin):
#         if row[0] in ppns:
#             write_to.writerow(row)

In [16]:
# commented code only needs to be run once

main = pd.read_csv(path+'/data/clean_data/main_prop14.csv', dtype=object)

main = main.drop_duplicates()
main = main[main.parcel.isin(ppns)].groupby('parcel').first()

main.to_csv(path+'/data/clean_data/main_prop_filtered.csv')

### Residential characteristics 
Filename: ```res2014.csv```

In [6]:
res = pd.read_csv(path+'/data/clean_data/res.csv')
res = res.loc[res.parcel.isin(ppns),['parcel','cqual','occup','rnumstor','year']]
res.to_csv(path+'/data/clean_data/res.csv', index=False)

### Tax bill
Filename: ```dec14_tci.csv```

In [ ]:
# ##only needs to be run once to clean taxbill data

# infile = path+'/data/original_data/taxbill/may15.csv'
# outfile = path+'/data/clean_data/taxbill_may15.csv'

# with open(infile, 'r') as fin, open(outfile, 'w') as fout:
#     write_to = csv.writer(fout, lineterminator='\n')
#     header = next(csv.reader(fin))
#     write_to.writerow(header)
#     for row in csv.reader(fin):
#         if row[5] in ppns:
#             write_to.writerow(row)

In [9]:
tb = pd.read_csv(path+'/data/clean_data/taxbill_may15.csv')

//anaconda/envs/data_sci/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4,20,22,137) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
cols = ['PROPERTY_NUMBER','TOTAL_NET_DELQ_BALANCE','TAX_ASSESSED_LAND','TAX_MARKET_LAND',\
        'LENDER_PROCESS_TYPE','GRAND_TOTAL_BALANCE','GRAND_TOTAL_PAID','GRAND_TOTAL_OWED']
tb[cols].to_csv(path+'/data/clean_data/taxbill_may15.csv',index=False)

### County land bank
Filename: ```count_land_bank.csv```

In [ ]:
lb = pd.read_csv(path+'/data/original_data/count_land_bank.csv', parse_dates=[3,4])
lb = lb[lb['parcel'].isin(ppns)]
lb.to_csv(path+'/data/clean_data/county_lb.csv', index=False)

### Foreclosure filings

In [8]:
fc = pd.read_csv(path+'/data/original_data/foreclosure_filings2006_beyond.csv', parse_dates = [2])
fc = fc[fc['parcel'].isin(ppns)]
fc.to_csv(path+'/data/clean_data/foreclosure_filings2.csv',index=False)

### Sheriff auction
Filename: ```shf_aution_mar2000_dec2014.csv```

In [ ]:
sa = pd.read_csv(path+'/data/original_data/shf_aution_mar2000_dec2014.csv', parse_dates=[2], encoding="ISO-8859-1")
sa = sa[sa.parcel.isin(ppns)]
sa.to_csv(path+'/data/clean_data/sheriff_auction.csv', index=False)

### Transfers
Filename: ```transfers2000_2014.csv```

In [ ]:
# do stuff like ethis for bigger files
infile = path+'/data/original_data/transfers2000_2014.csv'
outfile = path+'/data/clean_data/transfers.csv'

with open(infile, 'r') as fin, open(outfile, 'w') as fout:
    write_to = csv.writer(fout, lineterminator='\n')
    header = next(csv.reader(fin))
    write_to.writerow(header)
    for row in csv.reader(fin):
        if row[5] in ppns:
            write_to.writerow(row)

### Armslength sales
Filename: ```armslengthsales2006_2014.csv```

In [ ]:
al = pd.read_csv(path+'/data/original_data/armslengthsales2006_beyond.csv', dtype=str)
al = al[al.PROPERTY_NUMBER.isin(ppns)]
al.to_csv(path+'/data/clean_data/armslength.csv',index=False)

### Violations
Filename: ```violate_cle.csv```


In [ ]:
v = pd.read_csv(path+'/data/original_data/violate_cle.csv', dtype=str)
v = v[v.parcel.isin(ppns)]
v.to_csv(path+'/data/clean_data/violations.csv',index=False)

### Complaints
Filename: ```complaint_cle.csv```

In [2]:
c = pd.read_csv(path+'/data/original_data/complaint_cle.csv', dtype=str)
c = c[c.parcel.isin(ppns)]
c.to_csv(path+'/data/clean_data/complaints.csv')

### Postal data

In [ ]:
p = pd.DataFrame()
for pv in os.listdir(path+'/data/original_data/postal/'):
    pos = pd.read_csv(path+'/data/original_data/postal/' + pv)
    pos['date'] = dt.datetime(int(pv[2:6]), int(pv[6:8]), 1)
    p = p.append(pos)
p = p[p.PARCEL.isin(ppns)]
p.to_csv(path+'/data/clean_data/postal_vacancy.csv')